# EBAC - Regressão II - regressão múltipla

## Tarefa I

#### Previsão de renda II

Vamos continuar trabalhando com a base 'previsao_de_renda.csv', que é a base do seu próximo projeto. Vamos usar os recursos que vimos até aqui nesta base.

|variavel|descrição|
|-|-|
|data_ref                | Data de referência de coleta das variáveis |
|index                   | Código de identificação do cliente|
|sexo                    | Sexo do cliente|
|posse_de_veiculo        | Indica se o cliente possui veículo|
|posse_de_imovel         | Indica se o cliente possui imóvel|
|qtd_filhos              | Quantidade de filhos do cliente|
|tipo_renda              | Tipo de renda do cliente|
|educacao                | Grau de instrução do cliente|
|estado_civil            | Estado civil do cliente|
|tipo_residencia         | Tipo de residência do cliente (própria, alugada etc)|
|idade                   | Idade do cliente|
|tempo_emprego           | Tempo no emprego atual|
|qt_pessoas_residencia   | Quantidade de pessoas que moram na residência|
|renda                   | Renda em reais|

In [1]:
import pandas as pd
import seaborn as sns
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.metrics import r2_score

import statsmodels.api as sm
import statsmodels.formula.api as smf
import patsy


In [2]:
df = pd.read_csv('previsao_de_renda.csv')
df.head()

,Unnamed: 0,data_ref,id_cliente,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,0,2015-01-01,15056,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,1,2015-01-01,9968,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,2,2015-01-01,4312,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,3,2015-01-01,10639,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,4,2015-01-01,7064,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15000 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             15000 non-null  int64  
 1   data_ref               15000 non-null  object 
 2   id_cliente             15000 non-null  int64  
 3   sexo                   15000 non-null  object 
 4   posse_de_veiculo       15000 non-null  bool   
 5   posse_de_imovel        15000 non-null  bool   
 6   qtd_filhos             15000 non-null  int64  
 7   tipo_renda             15000 non-null  object 
 8   educacao               15000 non-null  object 
 9   estado_civil           15000 non-null  object 
 10  tipo_residencia        15000 non-null  object 
 11  idade                  15000 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  15000 non-null  float64
 14  renda                  15000 non-null  float64
dtypes:

In [4]:
#Exclusao dos dados nulos
df1 = df.dropna()
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 12427 entries, 0 to 14999
Data columns (total 15 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             12427 non-null  int64  
 1   data_ref               12427 non-null  object 
 2   id_cliente             12427 non-null  int64  
 3   sexo                   12427 non-null  object 
 4   posse_de_veiculo       12427 non-null  bool   
 5   posse_de_imovel        12427 non-null  bool   
 6   qtd_filhos             12427 non-null  int64  
 7   tipo_renda             12427 non-null  object 
 8   educacao               12427 non-null  object 
 9   estado_civil           12427 non-null  object 
 10  tipo_residencia        12427 non-null  object 
 11  idade                  12427 non-null  int64  
 12  tempo_emprego          12427 non-null  float64
 13  qt_pessoas_residencia  12427 non-null  float64
 14  renda                  12427 non-null  float64
dtypes: bool

In [5]:
#Exclusao das variaveis irrelevantes a analise
df1 = df1.drop(columns=['Unnamed: 0', 'data_ref', 'id_cliente'])
df1.head()

,sexo,posse_de_veiculo,posse_de_imovel,qtd_filhos,tipo_renda,educacao,estado_civil,tipo_residencia,idade,tempo_emprego,qt_pessoas_residencia,renda
0,F,False,True,0,Empresário,Secundário,Solteiro,Casa,26,6.602740,1.0,8060.34
1,M,True,True,0,Assalariado,Superior completo,Casado,Casa,28,7.183562,2.0,1852.15
2,F,True,True,0,Empresário,Superior completo,Casado,Casa,35,0.838356,2.0,2253.89
3,F,False,True,1,Servidor público,Superior completo,Casado,Casa,30,4.846575,3.0,6600.77
4,M,True,False,0,Assalariado,Secundário,Solteiro,Governamental,33,4.293151,1.0,6475.97


1. Separe a base em treinamento e teste (25% para teste, 75% para treinamento).
2. Rode uma regularização *ridge* com alpha = [0, 0.001, 0.005, 0.01, 0.05, 0.1] e avalie o $R^2$ na base de testes. Qual o melhor modelo?
3. Faça o mesmo que no passo 2, com uma regressão *LASSO*. Qual método chega a um melhor resultado?
4. Rode um modelo *stepwise*. Avalie o $R^2$ na vase de testes. Qual o melhor resultado?
5. Compare os parâmetros e avalie eventuais diferenças. Qual modelo você acha o melhor de todos?
6. Partindo dos modelos que você ajustou, tente melhorar o $R^2$ na base de testes. Use a criatividade, veja se consegue inserir alguma transformação ou combinação de variáveis.
7. Ajuste uma árvore de regressão e veja se consegue um $R^2$ melhor com ela.

In [6]:
print(df1.columns)

Index(['sexo', 'posse_de_veiculo', 'posse_de_imovel', 'qtd_filhos',
       'tipo_renda', 'educacao', 'estado_civil', 'tipo_residencia', 'idade',
       'tempo_emprego', 'qt_pessoas_residencia', 'renda'],
      dtype='object')


In [7]:
#1. Separaçao de base em treinamento e teste (25% teste, 75% treinamento)
X = df1.drop(columns=['renda'])
X = pd.get_dummies(X, drop_first=True)
X[['posse_de_veiculo', 'posse_de_imovel']] = X[['posse_de_veiculo','posse_de_imovel']].astype(int)

y = df1['renda']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=100)

In [8]:
#2. Regularização ridge com alpha = [0]
alpha = 0
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2_ridge}")

Alpha: 0 - R²: 0.2925875510859858


In [9]:
# Regularização ridge com alpha = [0.001]
alpha = 0.001
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2_ridge}")

Alpha: 0.001 - R²: 0.29258756799265095


In [10]:
# Regularização ridge com alpha = [0.005]
alpha = 0.005
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2_ridge}")

Alpha: 0.005 - R²: 0.29258763551736733


In [11]:
# Regularização ridge com alpha = [0.01]
alpha = 0.01
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2_ridge}")

Alpha: 0.01 - R²: 0.2925877196947876


In [12]:
# Regularização ridge com alpha = [0.05]
alpha = 0.05
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2_ridge}")

Alpha: 0.05 - R²: 0.2925883841482245


In [13]:
# Regularização ridge com alpha = [0.1]
alpha = 0.1
ridge = Ridge(alpha)
ridge.fit(X_train, y_train)

# Predizendo os valores para o conjunto de teste
y_pred = ridge.predict(X_test)

# Calculando o R² (coeficiente de determinação)
r2_ridge = r2_score(y_test, y_pred)
print(f"Alpha: {alpha} - R²: {r2_ridge}")

Alpha: 0.1 - R²: 0.29258919311073694


**Resultados de regularização Ridge**:

- Alpha: 0 - R²: 0.2925875510859858
- Alpha: 0.001 - R²: 0.29258756799265095
- Alpha: 0.005 - R²: 0.29258763551736733
- Alpha: 0.01 - R²: 0.2925877196947876
- Alpha: 0.1 - R²: 0.29258919311073694

**Conclusão**:
Todos os resultados são bem proximos e considerando as duas casas decimais, podemos dizer que sao iguais.


In [14]:
#3. Regularização LASSO
r2_scores_lasso = []
alphas = [0, 0.001, 0.005, 0.01, 0.05, 0.1] 

for alpha in alphas:
    lasso = Lasso(alpha=alpha)
    lasso.fit(X_train, y_train)
    
    y_pred_lasso = lasso.predict(X_test)
    r2_lasso = r2_score(y_test, y_pred_lasso)
    r2_scores_lasso.append((alpha, r2_lasso))

for alpha, r2_lasso in r2_scores_lasso:
    print(f'LASSO - Alpha: {alpha}, R²: {r2_lasso}')

C:\Users\AJ\anaconda3\lib\site-packages\sklearn\base.py:1151: UserWarning: With alpha=0, this algorithm does not converge well. You are advised to use the LinearRegression estimator
  return fit_method(estimator, *args, **kwargs)
C:\Users\AJ\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: UserWarning: Coordinate descent with no regularization may lead to unexpected results and is discouraged.
  model = cd_fast.enet_coordinate_descent(
C:\Users\AJ\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.968e+11, tolerance: 7.878e+07 Linear regression models with null weight for the l1 regularization term are more efficiently fitted using one of the solvers implemented in sklearn.linear_model.Ridge/RidgeCV instead.
  model = cd_fast.enet_coordinate_descent

LASSO - Alpha: 0, R²: 0.292587610958313
LASSO - Alpha: 0.001, R²: 0.29258786377259827
LASSO - Alpha: 0.005, R²: 0.2925888708966736
LASSO - Alpha: 0.01, R²: 0.29259012050236755
LASSO - Alpha: 0.05, R²: 0.2925997453719176
LASSO - Alpha: 0.1, R²: 0.2926108465188493


C:\Users\AJ\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:628: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.301e+10, tolerance: 7.878e+07
  model = cd_fast.enet_coordinate_descent(


**Conclusão da Regularizaçao LASSO**:
- O R² é o mesmo para todos os alphas testados: 0.292
- O R² da Regressao LASSO é praticamente igual ao RIDGE

In [15]:
#4. Modelo Stepwise
def stepwise_selection(X, y,
                       initial_list=[],
                       threshold_in=0.01,
                       threshold_out = 0.05,
                       verbose=True):
    """ Perform a forward-backward feature selection
    based on p-value from statsmodels.api.OLS
    Arguments:
        X - pandas.DataFrame with candidate features
        y - list-like with the target
        initial_list - list of features to start with (column names of X)
        threshold_in - include a feature if its p-value < threshold_in
        threshold_out - exclude a feature if its p-value > threshold_out
        verbose - whether to print the sequence of inclusions and exclusions
    Returns: list of selected features
    Always set threshold_in < threshold_out to avoid infinite looping.
    See https://en.wikipedia.org/wiki/Stepwise_regression for the details
    """
    included = list(initial_list)
    while True:
        changed=False
        # forward step
        excluded = list(set(X.columns)-set(included))
        new_pval = pd.Series(index=excluded)
        for new_column in excluded:
            print(included+[new_column])
            model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included+[new_column]]))).fit()
            new_pval[new_column] = model.pvalues[new_column]
        best_pval = new_pval.min()
        if best_pval < threshold_in:
            best_feature = new_pval.idxmin()
            included.append(best_feature)
            changed=True
            if verbose:
                print('Add  {:30} with p-value {:.6}'.format(best_feature, best_pval))

        # backward step
        print(included)
        model = sm.OLS(y, sm.add_constant(pd.DataFrame(X[included]))).fit()
        # use all coefs except intercept
        pvalues = model.pvalues.iloc[1:]
        worst_pval = pvalues.max() # null if pvalues is empty
        if worst_pval > threshold_out:
            changed=True
            worst_feature = pvalues.idxmax()
            included.remove(worst_feature)
            if verbose:
                print('Drop {:30} with p-value {:.6}'.format(worst_feature, worst_pval))
        if not changed:
            break
    return included

In [16]:
# Passo 1: Converter booleanos para numéricos
X_train = X_train.astype(float)

# Passo 2: Adicionar constante e criar o modelo
selected_features = stepwise_selection(X_train, y_train, verbose=True)

# Verifique se a seleção retornou colunas válidas
if selected_features:
    X_selected = sm.add_constant(X_train[selected_features])
    reg_stepwise = sm.OLS(y_train, X_selected).fit()
    print(reg_stepwise.summary())
else:
    print("Nenhuma variável foi selecionada.")


['tipo_residencia_Com os pais']
['tipo_residencia_Estúdio']
['educacao_Pós graduação']
['idade']
['educacao_Superior incompleto']
['tempo_emprego']
['qt_pessoas_residencia']
['educacao_Secundário']
['educacao_Superior completo']
['posse_de_imovel']
['posse_de_veiculo']
['tipo_renda_Empresário']
['tipo_renda_Servidor público']
['sexo_M']
['tipo_residencia_Comunitário']
['tipo_renda_Pensionista']
['tipo_residencia_Casa']
['tipo_residencia_Governamental']
['estado_civil_União']
['estado_civil_Viúvo']
['estado_civil_Separado']
['qtd_filhos']
['estado_civil_Solteiro']
['tipo_renda_Bolsista']
Add  tempo_emprego                  with p-value 0.0
['tempo_emprego']
['tempo_emprego', 'tipo_residencia_Com os pais']
['tempo_emprego', 'tipo_residencia_Estúdio']
['tempo_emprego', 'educacao_Pós graduação']
['tempo_emprego', 'idade']
['tempo_emprego', 'educacao_Superior incompleto']
['tempo_emprego', 'qt_pessoas_residencia']
['tempo_emprego', 'educacao_Secundário']
['tempo_emprego', 'educacao_Superior

**Conclusões sobre o stepwise:**
- O modelo stepwise gerou um R² = 0.24, que é menor que a regularizacao Ridge e Lasso.

***5. Comparação de todos os modelos***:

In [17]:
# Resultado do modelo Ridge
print(f"Coeficientes Ridge: {ridge.coef_}")
print(f"R² Ridge: {ridge.score(X_test, y_test)}")


Coeficientes Ridge: [  131.57317641   300.12454885 -1185.95083731    40.29693496
   566.21514209  1176.17794069  6096.22445607 -1413.73345871
   735.71342646 -2154.84071643   -79.72621667  1582.25007947
   990.07655823  1403.20619081   699.92824766  1087.08207467
   682.43559753  -384.01505022  1000.54620907  -394.93307421
  -427.09236597  -564.57652767   370.93285896  -258.65109852]
R² Ridge: 0.29258919311073694


In [18]:
# Resultado do modelo Lasso
print(f"Coeficientes Lasso: {lasso.coef_}")
print(f"R² Lasso: {lasso.score(X_test, y_test)}")

Coeficientes Lasso: [  131.37046382   300.08426706 -1086.81461825    40.31180987
   566.19627161  1077.00653248  6096.11134324 -1215.35855493
   736.00490345 -2064.4991642    -77.73811511  1507.29121974
   950.4218027   1362.70433943   658.5278353    986.20121259
   583.46381487  -383.11675888   896.21301483  -375.32644627
  -405.91957033  -529.45989359   372.98497519  -235.45014909]
R² Lasso: 0.2926108465188493


In [19]:
print(f"Coeficientes Stepwise: {reg_stepwise.params}")
print(f"R² Stepwise: {reg_stepwise.rsquared}")

Coeficientes Stepwise: const                        -2711.399449
tempo_emprego                  564.577990
sexo_M                        6173.508633
idade                           44.977910
tipo_renda_Empresário          740.131210
educacao_Superior completo     456.439221
dtype: float64
R² Stepwise: 0.24551857731216875


In [20]:
#6.Transformacao da renda em logaritmo para melhorar o R²
modelo = 'np.log(renda) ~ sexo + posse_de_veiculo + posse_de_imovel + qtd_filhos + tipo_renda + educacao + estado_civil + tipo_residencia + idade + tempo_emprego + qt_pessoas_residencia'
md = smf.ols(modelo, data = df1)
reg_lasso = md.fit_regularized(method = 'elastic_net'
                        , refit = True
                        , L1_wt = 1 
                        , alpha = 0)
reg_lasso.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          np.log(renda)   R-squared:                       0.357
Model:                            OLS   Adj. R-squared:                  0.356
Method:                 Least Squares   F-statistic:                     276.0
Date:                Mon, 13 Jan 2025   Prob (F-statistic):               0.00
Time:                        23:39:12   Log-Likelihood:                -13568.
No. Observations:               12427   AIC:                         2.719e+04
Df Residuals:                   12402   BIC:                         2.738e+04
Df Model:                          25                                         
Covariance Type:            nonrobust                                         
====================================================================================================
                                       coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------------------
Intercept                            6.5847      0.235     28.006      0.000       6.124       7.046
sexo[T.M]                            0.7874      0.015     53.723      0.000       0.759       0.816
posse_de_veiculo[T.True]             0.0441      0.014      3.119      0.002       0.016       0.072
posse_de_imovel[T.True]              0.0829      0.014      5.926      0.000       0.055       0.110
tipo_renda[T.Bolsista]               0.2209      0.241      0.916      0.360      -0.252       0.694
tipo_renda[T.Empresário]             0.1551      0.015     10.387      0.000       0.126       0.184
tipo_renda[T.Pensionista]           -0.3087      0.241     -1.280      0.201      -0.782       0.164
tipo_renda[T.Servidor público]       0.0576      0.022      2.591      0.010       0.014       0.101
educacao[T.Pós graduação]            0.1071      0.159      0.673      0.501      -0.205       0.419
educacao[T.Secundário]              -0.0141      0.072     -0.196      0.844      -0.155       0.127
educacao[T.Superior completo]        0.0938      0.072      1.298      0.194      -0.048       0.235
educacao[T.Superior incompleto]     -0.0432      0.078     -0.555      0.579      -0.196       0.109
estado_civil[T.Separado]             0.3265      0.111      2.935      0.003       0.108       0.545
estado_civil[T.Solteiro]             0.2672      0.109      2.453      0.014       0.054       0.481
estado_civil[T.União]               -0.0343      0.025     -1.361      0.174      -0.084       0.015
estado_civil[T.Viúvo]                0.3689      0.116      3.191      0.001       0.142       0.596
tipo_residencia[T.Casa]             -0.0442      0.054     -0.815      0.415      -0.150       0.062
tipo_residencia[T.Com os pais]      -0.0258      0.060     -0.427      0.670      -0.144       0.093
tipo_residencia[T.Comunitário]      -0.1196      0.105     -1.135      0.256      -0.326       0.087
tipo_residencia[T.Estúdio]           0.0665      0.099      0.670      0.503      -0.128       0.261
tipo_residencia[T.Governamental]    -0.0571      0.066     -0.865      0.387      -0.186       0.072
qtd_filhos                          -0.2658      0.108     -2.463      0.014      -0.477      -0.054
idade                                0.0053      0.001      6.624      0.000       0.004       0.007
tempo_emprego                        0.0617      0.001     59.398      0.000       0.060       0.064
qt_pessoas_residencia                0.2926      0.108      2.716      0.007       0.081       0.504
==============================================================================
Omnibus:                        0.858   Durbin-Watson:                   2.023
Prob(Omnibus):                  0.651   Jarque-Bera (JB):                0.839
Skew:             

**Transformando a variavel 'renda' em logaritmo, obtivemos um aumento no R² para 0.357**

In [21]:
#7. Criando e ajustando a árvore de regressão
tree_model = DecisionTreeRegressor(random_state=100)
tree_model.fit(X_train, y_train)

# Avaliando o modelo na base de teste
y_pred_tree = tree_model.predict(X_test)
mse_tree = mean_squared_error(y_test, y_pred_tree)
r2_tree = tree_model.score(X_test, y_test)

print(f"R² Árvore de Regressão: {r2_tree}")

R² Árvore de Regressão: 0.39153094245802644


O R² da Arvore de Regressao foi maior que todos os modelos: 0.391